In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [6]:
def get_links(url):
    website=Website(url)
    ollama_via_openai=OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response=ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [7]:
get_links('https://github.com')

{'links': ['https://github.com/features/copilot',
  'https://github.com/features security',
  'https://github.com/features/actions',
  'https://github.com/features/codespaces',
  'https://github.com/features/issues',
  'https://github.com/features/code-review',
  'https://docs.github.com',
  'https://skills.github.com',
  'https://github.blog',
  'https://github.com/team',
  'https://github.com/enterprise',
  'https://resources.github.com',
  'https://resources.github.com/learn/pathways',
  'https://resource/github.com',
  'https://github.com/customer-stories',
  'https://partner.github.com',
  'https://github.com/solutions/executive-insights',
  'https://docs.github.com/site-policy/github-terms-of-service',
  'https://docs.github.com/site-policy/privacy-policies/github-privacy-statement',
  'https://www.linkedin.com/company/github',
  'https://www.instagram.com/github',
  'https://www.youtube.com/github']}

In [8]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [9]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [10]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [11]:
def create_brochure(company_name, url):
    ollama_via_openai=OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [12]:
def stream_brochure(company_name, url):
    ollama_via_openai=OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [14]:
stream_brochure("GitHub", "https://github.com/")

Found links: {'relevant_links': ['https://github.com/features/copilot', 'https://github.com/features/security', 'https://github.com/features/actions', 'https://github.com/features/codespaces', 'https://github.com/features/issues', 'https://github.com/features/code-review', 'https://github.com/features/discussions', 'https://skills.github.com']}


**GitHub: Revolutionizing Software Development**

Welcome to the world's most widely adopted AI-powered developer platform, GitHub!

**About Us**

At GitHub, we believe that software development should be collaborative, secure, and efficient. Our mission is to empower developers to build and ship software faster, with less frustration. We're a community of millions of developers who share, collaborate, and learn from each other.

**What We Do**

We offer a comprehensive set of tools to help you manage your codebase, automate workflows, and secure your projects. Our flagship product, GitHub Copilot, uses AI to assist developers in writing better code, faster.

**Key Features**

* **GitHub Copilot**: Our AI-powered code assistant helps with code completion, chat, and more.
* **Codespaces**: Instant dev environments in the cloud, so you can start building and coding anywhere, anytime.
* **GitHub Actions**: Simplify your CI/CD process with our easy-to-use platform.

**Who We Help**

We've helped businesses of all sizes, from small startups to large enterprises, accelerate their performance and improve developer productivity. Our customers have seen significant increases in speed, security, and efficiency.

**How You Can Join Us**

Are you interested in joining the GitHub community? Here's what we're looking for:

* **Developers**: We need your coding skills and passion to build and contribute to our platform.
* **Engineers**: We're always on the lookout for talented engineers to help us grow our products and features.
* **Entrepreneurs**: If you have an idea for a new product or service, we'd love to hear from you.

**Stay Connected**

Want to stay up-to-date with the latest news, tips, and behind-the-scenes stories from GitHub? Follow us on:

[Website](https://github.com)
[LinkedIn](https://www.linkedin.com/company/github)
[Twitter](https://twitter.com/github)
[Facebook](https://facebook.com/github)